[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PiotrTyrakowski/PolishLawLLM-Benchmark/blob/KwiatkowskiML/bielik-notebook/notebooks/bielik.ipynb)


In [ ]:
from benchmark_framework.models.bielik import BielikModel

rm -fr PolishLawLLM-Benchmark/

In [ ]:
!git clone -b KwiatkowskiML/bielik-notebook https://github.com/PiotrTyrakowski/PolishLawLLM-Benchmark.git

In [ ]:
%cd PolishLawLLM-Benchmark/
!pip install -e .

In [ ]:
!pip install -q transformers torch accelerate sentencepiece

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

In [ ]:
from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('HF_TOKEN'))

model_name = "speakleash/Bielik-4.5B-v3.0-Instruct"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

In [ ]:
prompt = "Przepis na jajecznicę w dwóch zdaniach"
messages = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
outputs = model.generate(input_ids, max_new_tokens=1000)
response = tokenizer.batch_decode(outputs)[0]
print(response)

In [ ]:
import json
from google.colab import drive
from benchmark_framework.models.bielik import BielikModel

drive.mount('/content/drive')
exam_path = '/content/drive/MyDrive/example_exams/2024.jsonl'

bielik = BielikModel("speakleash/Bielik-11B-v2.2-Instruct")

# with open(exam_path, 'r', encoding='utf-8') as f:
#     for line in f:
#         try:
#             obj = json.loads(line)
#             print(obj)
#         except json.JSONDecodeError:
#             print(line.rstrip())
